## Weighted Regression in `python`                                   :jupyter:



##### Problem set 1, section 5
##### Description: Adaptation of code in weighted_regression.ipynb 
##### Group: Bobing, Cassandra, Max, Prema, Rajdev, Yazen 

#### Goal: Extend the code to actually estimate beta in the case where k=3 and y=Xbeta+u with ET'u=0 and y is a Nxk matrix.

The fact that $T$ and $u$ are &ldquo;independent&rdquo; (or at least
orthogonal) variables means that if we want to compute a
&ldquo;classical&rdquo; regression we&rsquo;d do it something like this:



##### Define independent random variables



In [88]:
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal

k = 3 # Number of observables in T

mu = [0]*k

## CHANGE 1: MAKE SIGMA A DIAGONAL MATRIX 
Sigma=[[1,0,0],
       [0,2,0],
       [0,0,3]]

T = multivariate_normal(mu,Sigma)

# CHANGE 2: CHANGE U 
u = multivariate_normal(mu,Sigma)
# u = multivariate_normal(cov=0.2)


##### Define `X`



Recall that $X$ can depend on $T$ and $u$.  This dependence needn&rsquo;t be
linear!  For example, suppose $X=T^3D + u$, where $D$ is an
$\ell\times k$ matrix.



##### Construct Sample



To construct a sample of observables $(y,X,T)$ we just use the regression equation,
      plus an assumption about the value of $\beta$:



In [89]:
# CHANGE 3: MAKE BETA 3X3 
beta = [[1/2,1,2],
        [2,3,4],
        [1,0,6]]

# CHANGE 4: MAKE D 3X3
D = np.random.random(size=(3,3)) # Generate random 3x2 matrix

N=1000 # Sample size

# Now: Transform rvs into a sample
T = T.rvs(N)

u = u.rvs(N) # Replace u with a sample

X = (T**3)@D  # Note use of ** operator for exponentiation

y = X@beta + u # Note use of @ operator for matrix multiplication


##### Turn to estimation



So, we now have data on *realizations* $(y,X,T)$.  Now forget
     that we know $\beta$ and let&rsquo;s estimate it, using weighted least
     squares.  As a numerical matter it&rsquo;s better to avoid explicitly
     inverting the $(T^T X)$ matrix; instead we can solve the &ldquo;normal&rdquo;
     equations

\begin{align*}
   T'y &= T' X b + T' u\\
   \mbox{E}(T'u) = 0
\end{align*}



##### Numerical solution



In the classical case we were trying to solve a linear system that
 took the form $Ab=0$, with $A$ a square matrix.  In the present case
 we&rsquo;re also trying to solve a linear system, but with a matrix $A$
 that may have more rows than columns.  Provided the rows are linearly
 independent, this implies that we have an **overidentified** system of
 equations.  We&rsquo;ll return to the implications of this later, but for
 now this also calls for a different numerical approach, using
 `np.linalg.lstsq` instead of `np.linalg.solve`.



In [90]:
from scipy.linalg import inv, sqrtm

b = np.linalg.lstsq(T.T@X,T.T@y, rcond=None)[0] # lstsqs returns several results

e = y - X@b

print(b)

TXplus = np.linalg.pinv(T.T@X) # Moore-Penrose pseudo-inverse

# Covariance matrix of b
vb = e.var()*TXplus@T.T@T@TXplus.T  # u is known to be homoskedastic

print(vb)

[[ 0.50240756  0.98104413  1.9787112 ]
 [ 1.98411949  3.05918599  4.03856139]
 [ 0.98225217 -0.04598303  5.98161753]]
[[ 0.00057104 -0.00169626  0.00112185]
 [-0.00169626  0.00529417 -0.00379543]
 [ 0.00112185 -0.00379543  0.00350825]]
